# Working with CFG Parsers

## Getting Started

This assignment requires that you have set up the Stanford parser. First, make sure you have the more recent version of [Java](https://www.java.com/en/download/), then get the [Stanford CoreNLP](https://stanfordnlp.github.io/CoreNLP/) package. Make sure that you get the 4.3.2 version (or at least >=4.2.0). 

To load Stanford CoreNLP in Python, change the `coreNLP_dir` variable in the code below to where you unzipped Stanford coreNLP. You can follow this [tutorial](https://bbengfort.github.io/snippets/2018/06/22/corenlp-nltk-parses.html). Once the coreNLP server is running, you will be able to access it through NLTK.

It may take a few seconds or up to a minute to start the server.

In [8]:
from nltk.parse.corenlp import CoreNLPServer
import os
import time


coreNLP_dir = "/Users/varadrajrameshpoojary/Downloads/stanford-corenlp-4.3.2" # Change this to your coreNLP directory

server = CoreNLPServer(
   os.path.join(coreNLP_dir, "stanford-corenlp-4.3.2.jar"),
   os.path.join(coreNLP_dir, "stanford-corenlp-4.3.2-models.jar")    
)
server.start()

parsed a sentence to make sure that everything works.

In [9]:
from nltk.parse.corenlp import CoreNLPParser

parser = CoreNLPParser()
parse = next(parser.raw_parse("I put the book in the box on the table."))
print(parse)

(ROOT
  (S
    (NP (PRP I))
    (VP
      (VBD put)
      (NP (DT the) (NN book))
      (PP (IN in) (NP (DT the) (NN box)))
      (PP (IN on) (NP (DT the) (NN table))))
    (. .)))


In [3]:
parse = next(parser.raw_parse("They gave me yellow and blue pants"))
print(parse)

(ROOT
  (S
    (NP (PRP They))
    (VP
      (VBD gave)
      (NP (PRP me))
      (NP (ADJP (JJ yellow) (CC and) (JJ blue)) (NNS pants)))))


Run the code below if you want to shut down the coreNLP server after you've finished with it. It's a good idea to shut down the parser after finishing work with it because it may remain running in the background and you may not be able to start another parser instance without restarting your computer or manually killing the parser process. 

If you forget to stop the server, next time when you try to launch it, you'll get an error. In this case, you may first need to kill the old server manually. To do this, you can run `ps -ax | grep stanford` on the commandline (at least on OSX and Linux) which should give you the process ID of the server, e.g. 11111. You can then use `kill -9 11111` to kill the parser, after which you should be able to start the server again.

In [4]:
try:
    server.stop()
except:
    pass

Other things you'll need:

In [5]:
import nltk
from nltk.tree import Tree
from nltk.grammar import CFG,Nonterminal,Production,FeatureGrammar
from nltk.parse import CoreNLPParser
from nltk.corpus import brown

nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/varadrajrameshpoojary/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

We then parse the first 20 sentences of the NLTK `movie_reviews` corpus, and report the average depth (height) of the parse trees.

To retain the tokenization in the `move_reviews` corpus. You can use `parser.parse()` to parse the tokenized input sentences. Note that `parser.parse()` returns an iterator over possible parses. There may be several if the sentence is ambiguous. You can compute statistics on the first sentence which is returned by `parser.parse()`.

In [10]:
from nltk.corpus import movie_reviews
# your code here
test_set= movie_reviews.sents()[:20]
height_list = []
for sentence in test_set:
    try:
        parse = next(parser.parse(sentence))
        height_list.append(parse.height())
        
    except Exception:
        pass 
print(sum(height_list)/len(height_list))

11.7


### Build a gold standard

One typical way to build treebanks is, rather than having humans build a tree from scratch, instead use an automatic parser to give an initial parse, and then have humans do a second pass to fix any errors. 

We will use the following three sentences from the NLTK `movie_reviews` corpus to build a mini-treebank:

In [11]:
corpus = [['oh', ',', 'and', 'by', 'the', 'way', ',', 'this', 'is', 'not', 'a', 'horror', 'or', 'teen', 'slasher', 'flick', '.'],
          ['little', 'do', 'they', 'know', 'the', 'power', 'within', '.'],
          ['so', ',', 'if', 'robots', 'and', 'body', 'parts', 'really', 'turn', 'you', 'on', ',', 'here', "'", 's', 'your', 'movie', '.']]

In each of the three sections below you will see a parse tree produced by CorNLPParser. Each of the trees contains at least one parse error.  

Create an NLTK Tree corresponding to the correct parse, which should be appended to the `gold_standard_parses` list below. You can do this manually by printing the tree, creating a triple-quoted string, modifying it, and converting it back into a `Tree` using the function `Tree.fromstring` following the example below. 

In [12]:
# Example:

# The second phrase should be an VP, not an NP
err_tree_str = '''(S
(NP (NNS Dogs)) 
(NP (VBN bark))
(. .))
'''

corr_tree_str = '''(S
(NP (NNS Dogs)) 
(VP (VBN bark))
(. .))
'''

corr_tree = Tree.fromstring(corr_tree_str)
print(corr_tree)

(S (NP (NNS Dogs)) (VP (VBN bark)) (. .))


In [13]:
# Store your corrected nltk.Tree objects in this list
gold_standard_parses = []

#### Sentence 1

In [14]:
# The word "flick" should be modified by the entire noun phrase 
# "a horror or teen slasher" instead of just "teen slasher". A noun 
# phrase which modifies a noun is labeled as NML.
err_tree_str = '''(ROOT
  (S
    (INTJ (UH oh))
    (, ,)
    (CC and)
    (PP (IN by) (NP (DT the) (NN way)))
    (, ,)
    (NP (DT this))
    (VP
      (VBZ is)
      (RB not)
      (NP
        (NP (DT a) (NN horror))
        (CC or)
        (NP (NML (NN teen) (NN slasher)) (NN flick))))
    (. .)))'''

# your code here
corr_tree_str = '''(ROOT
  (S
    (INTJ (UH oh))
    (, ,)
    (CC and)
    (PP (IN by) (NP (DT the) (NN way)))
    (, ,)
    (NP (DT this))
    (VP
      (VBZ is)
      (RB not)
      (NP
        (NP (DT a) (NN horror)
        (CC or)
        (NML (NN teen) (NN slasher))) (NN flick))
    (. .))))'''
print("err_tree_str:")
list(Tree.fromstring(err_tree_str))[0].pretty_print()
print("corr_tree_str:")
list(Tree.fromstring(corr_tree_str))[0].pretty_print()

gold_standard_parses.append(Tree.fromstring(corr_tree_str))

err_tree_str:
                                       S                                                   
  _____________________________________|_________________________________________________   
 |    |   |       |           |   |                     VP                               | 
 |    |   |       |           |   |     ________________|_____                           |  
 |    |   |       |           |   |    |   |                  NP                         | 
 |    |   |       |           |   |    |   |        __________|______________            |  
 |    |   |       PP          |   |    |   |       |          |              NP          | 
 |    |   |    ___|___        |   |    |   |       |          |         _____|______     |  
INTJ  |   |   |       NP      |   NP   |   |       NP         |       NML           |    | 
 |    |   |   |    ___|___    |   |    |   |    ___|____      |    ____|_____       |    |  
 UH   ,   CC  IN  DT      NN  ,   DT  VBZ  RB  DT       NN   

#### Sentence 2

In [15]:
# The PP "within" should attach to the NP "the power", not to the VP "know the power". 
err_tree_str = '''(ROOT
  (S
    (NP (RB little))
    (VP
      (VBP do)
      (SBAR
        (S
          (NP (PRP they))
          (VP (VBP know) (NP (DT the) (NN power)) (PP (IN within))))))
    (. .)))'''

# your code here

corr_tree_str = '''(ROOT
  (S
    (NP (RB little))
    (VP
      (VBP do)
      (SBAR
        (S
          (NP (PRP they))
          (VP (VBP know) (NP (DT the) (NN power) (PP (IN within)))))))
    (. .)))'''
print("err_tree_str:")
list(Tree.fromstring(err_tree_str))[0].pretty_print()
print("corr_tree_str:")
list(Tree.fromstring(corr_tree_str))[0].pretty_print()

gold_standard_parses.append(Tree.fromstring(corr_tree_str))

err_tree_str:
                 S                           
   ______________|_________________________   
  |         VP                             | 
  |      ___|____                          |  
  |     |       SBAR                       | 
  |     |        |                         |  
  |     |        S                         | 
  |     |    ____|____                     |  
  |     |   |         VP                   | 
  |     |   |     ____|______________      |  
  NP    |   NP   |        NP         PP    | 
  |     |   |    |     ___|____      |     |  
  RB   VBP PRP  VBP   DT       NN    IN    . 
  |     |   |    |    |        |     |     |  
little  do they know the     power within  . 

corr_tree_str:
                 S                            
   ______________|__________________________   
  |              VP                         | 
  |      ________|____                      |  
  |     |            SBAR                   | 
  |     |             |             

#### Sentence 3

In [16]:
# There are several errors."body" and "parts" should form an NP. 
# Moreover, "here's your movie" should form a clause and "s" is in 
# fact a verb.  
err_tree_str = '''(ROOT
  (SINV
    (ADVP (RB so))
    (, ,)
    (PP
      (IN if)
      (NP (NML (NNS robots) (CC and) (NN body)) (NNS parts)))
    (ADVP (RB really))
    (VP
      (VBP turn)
      (NP (NP (PRP you)) (PP (IN on) (, ,) (NP (RB here))) ('' '))
      (S (NP (POS s))))
    (NP (PRP$ your) (NN movie))
    (. .)))'''

# your code here


corr_tree_str = '''(ROOT
  (SINV
    (ADVP (RB so))
    (, ,)
    (PP
      (IN if)
      (NP (NML (NNS robots) (CC and) (NP(NN body) (NNS parts))))
    (ADVP (RB really))
    (VP
      (VBP turn)
      (NP (NP (PRP you)) (PP (IN on))) (, ,) (CLAUSE (NP (RB here)) ('' ')
      (S (VB (POS s)))
    (NP (PRP$ your) (NN movie))
    (. .))))))'''
print("err_tree_str:")
list(Tree.fromstring(err_tree_str))[0].pretty_print()
print("corr_tree_str:")
list(Tree.fromstring(corr_tree_str))[0].pretty_print()


gold_standard_parses.append(Tree.fromstring(corr_tree_str))

err_tree_str:
                                              SINV                                            
  _____________________________________________|____________________________________________   
 |    |              |                   |                  VP                    |         | 
 |    |              |                   |      ____________|____________         |         |  
 |    |              PP                  |     |            NP           |        |         | 
 |    |    __________|___                |     |     _______|________    |        |         |  
 |    |   |              NP              |     |    |       PP       |   S        |         | 
 |    |   |           ___|_________      |     |    |    ___|___     |   |        |         |  
ADVP  |   |         NML            |    ADVP   |    NP  |   |   NP   |   NP       NP        | 
 |    |   |     _____|_______      |     |     |    |   |   |   |    |   |    ____|____     |  
 RB   ,   IN  NNS    CC      NN

### Evaluating parsers

Now that we have a gold standard, we can use it to evaluate parser output. 


We start by writing a function, get_constituents, which takes a parse tree and returns a set of tuples, where each tuple is (*label*, *start*, *end*) where *start* and *end* correspond to the indicies of a corresponding constituent (phrase) in the sentence and *label* is the label of that constituent. 

We do **not** include simple POS constituents `(POS word)` like `(VBD ate)`. We want to evaluate the parser only on actual phrases.


In [17]:
def get_constituents(tree,start_index=0):
    constituents = set()
    if tree.height() > 2:
        constituents.add((tree.label(), start_index, start_index+len(tree.leaves())))
        for node in tree:
            constituents.update(get_constituents(node, start_index))
            start_index += len(node.leaves())
    return constituents

In [18]:
tree = Tree.fromstring('''(S (NP (DT the) (DT mouse)) (VP (VBD ate) (NP (NP (DT the) (DT mouse)) (POS 's) (NN cheese))) )''')
assert get_constituents(tree) == {("S",0,7), ("NP",0,2), ("VP",2,7),("NP",3,7),("NP",3,5)}
tree = Tree.fromstring('''(S (NP (DET the) (NP (NN cat) (CC and) (NN dog))) (VP (VBD fought)))''')
assert get_constituents(tree) == {("S",0,5), ("NP",0,4), ("NP",1,4),("VP",4,5)}
print("Success!")

Success!


We write a function `parse_f1` which uses get_constituents to implement the constituent F-score measure. It should be given two lists, a lists of proposed parses and a corresponding list of gold standard parses, and return an F-score reflecting how close the proposed parses match. 



In [19]:
def parse_f1(proposed_parses,gold_parses):
    f1score = 0
    # your code here
    match_count = 0
    total_gold_count =0
    total_proposed_count =0
    for gold,proposed in zip(gold_parses,proposed_parses):
        gold_parse_set = get_constituents(gold)
        proposed_parse_set = get_constituents(proposed)
        match_count += len(gold_parse_set & proposed_parse_set)
        total_gold_count += len(gold_parse_set)
        total_proposed_count += len(proposed_parse_set )
    precision = match_count/total_proposed_count
    recall = match_count/total_gold_count
    f1score = 2*((precision*recall)/(precision+recall))
    print(f1score)
    return f1score

In [20]:
gold_parses = [Tree.fromstring('''(S (NP (DT the) (DT mouse)) (VP (VBD ate) (NP (NP (DT the) (DT mouse)) (POS 's) (NN cheese))) )'''), Tree.fromstring('''(S (NP (NNS mice)) (VP (VBD love) (NP (NNS ducks))))''')]
proposed_parses = [Tree.fromstring('''(S (NP (DT the) (DT mouse)) (VP (VBD ate) (NP (NP (DT the) (DT mouse)) (POS 's) (NN cheese))) )'''), Tree.fromstring('''(S (NP (NNS mice) (NN love)) (VP (VBZ ducks)))''')]
assert 0.71> parse_f1(proposed_parses,gold_parses) > 0.7
print("Success!")

0.7058823529411765
Success!


we parse our three example sentences from above using CoreNLPParser (you can find the sentences in the list `corpus`), extract the constituents and evaluate the result against `gold_standard_parses`. 

In [21]:
# your code here
parsed_corpus = [next(parser.parse(s)) for s in corpus]
print("Parser f1-score: %.2f" % parse_f1(parsed_corpus, gold_standard_parses))    

0.7123287671232876
Parser f1-score: 0.71


### Generate a grammar

Parse trees implicitly contain the production rules for a CFG defined by the productions which are present in the parse tree. You can access these productions using the member function `nltk.Tree.productions()`. 

We produce a grammar corresponding to the CFG productions in our three sentences, and print it out. 

In [22]:
corpus = [['oh', ',', 'and', 'by', 'the', 'way', ',', 'this', 'is', 'not', 'a', 'horror', 'or', 'teen', 'slasher', 'flick', '.'],
          ['little', 'do', 'they', 'know', 'the', 'power', 'within', '.'],
          ['so', ',', 'if', 'robots', 'and', 'body', 'parts', 'really', 'turn', 'you', 'on', ',', 'here', "'", 's', 'your', 'movie', '.']]

In [23]:
rules = set()

# your code here
for tree in gold_standard_parses:
    rule_list= tree.productions()
    for i in rule_list:
        rules.add(i)

for rule in rules:
    print(rule)

. -> '.'
DT -> 'this'
NN -> 'body'
DT -> 'a'
NP -> PRP$ NN
'' -> "'"
UH -> 'oh'
PRP -> 'you'
NN -> 'slasher'
S -> NP VP
IN -> 'within'
SINV -> ADVP , PP
RB -> 'so'
NP -> RB
NML -> NNS CC NP
PP -> IN
VP -> VBP NP , CLAUSE
VP -> VBP SBAR
DT -> 'the'
IN -> 'by'
NNS -> 'robots'
NN -> 'horror'
ROOT -> S
VBZ -> 'is'
NN -> 'movie'
NN -> 'flick'
VB -> POS
S -> INTJ , CC PP , NP VP
RB -> 'little'
NNS -> 'parts'
PP -> IN NP
VBP -> 'do'
NN -> 'power'
NML -> NN NN
NN -> 'way'
NP -> DT
PRP -> 'they'
VP -> VBP NP
ROOT -> SINV
NP -> NML
NP -> NN NNS
VBP -> 'turn'
SBAR -> S
IN -> 'on'
S -> NP VP .
NN -> 'teen'
POS -> 's'
RB -> 'really'
CC -> 'or'
PP -> IN NP ADVP VP
NP -> NP NN
PRP$ -> 'your'
ADVP -> RB
IN -> 'if'
NP -> PRP
INTJ -> UH
, -> ','
NP -> DT NN CC NML
NP -> DT NN PP
NP -> DT NN
RB -> 'here'
NP -> NP PP
CLAUSE -> NP '' S NP .
S -> VB
RB -> 'not'
VP -> VBZ RB NP .
VBP -> 'know'
CC -> 'and'



We show the rules are indeed sufficient to parse the sentences in the list `corpus`. Using an NLTK EarleyChartParser parser for this. Print out the number of parses for each sentence.

In [24]:
# your code here
from nltk.parse import EarleyChartParser

# your code here
nltk_parser = EarleyChartParser(CFG(Nonterminal('ROOT'), rules), trace=0)
    
for sent in corpus:
    print("Parses for %s:" % sent)
    parses = list(nltk_parser.parse(sent))
    print("Number of Parses : \n", len(parses))

Parses for ['oh', ',', 'and', 'by', 'the', 'way', ',', 'this', 'is', 'not', 'a', 'horror', 'or', 'teen', 'slasher', 'flick', '.']:
Number of Parses : 
 2
Parses for ['little', 'do', 'they', 'know', 'the', 'power', 'within', '.']:
Number of Parses : 
 6
Parses for ['so', ',', 'if', 'robots', 'and', 'body', 'parts', 'really', 'turn', 'you', 'on', ',', 'here', "'", 's', 'your', 'movie', '.']:
Number of Parses : 
 1
